In [ ]:
import tensorflow as tf
import sys
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import tqdm
import numpy as np
from tensorflow.python import metrics
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import drive
from keras.models import load_model
from keras.models import Sequential, Model
from sklearn import svm
from keras.models import load_model
import matplotlib.pyplot as plt
from tensorflow.python.ops.metrics import accuracy
import numpy
import sklearn.metrics as metrics
import math
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.svm import SVC
import docx

In [ ]:
from zipfile import ZipFile
file_name = "Dhad-Split.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:
train_datagen = ImageDataGenerator (
    rescale = 1./255.,
)
training_set = train_datagen.flow_from_directory (
    "/content/Dhad-Split/train",
    target_size=(32,32),
    batch_size=32,
    class_mode="categorical",
    color_mode="grayscale",
    shuffle=True,
    seed=42
)

# extract X and Y
training_set.reset()
X_train, y_train = next(training_set)
for i in tqdm.tqdm(range(int(training_set.n/32)-1)): 
  img, label = next(training_set)
  X_train = np.append(X_train, img, axis=0 )
  y_train = np.append(y_train, label, axis=0)
print(X_train.shape, y_train.shape)

In [ ]:
val_datagen = ImageDataGenerator (
    rescale=1./255,
)
validation_set = val_datagen.flow_from_directory (
    "/content/Dhad-Split/val",
    target_size=(32,32),
    batch_size=32,
    class_mode="categorical",
    color_mode="grayscale",
    shuffle=True,
    seed=42
)

# extract X and Y
validation_set.reset()
X_val, y_val = next(validation_set)
for i in tqdm.tqdm(range(int(validation_set.n/32)-1)): 
  img, label = next(validation_set)
  X_val = np.append(X_val, img, axis=0 )
  y_val = np.append(y_val, label, axis=0)
print(X_val.shape, y_val.shape)

In [ ]:
test_datagen = ImageDataGenerator (
    rescale=1./255,
)
testing_set = test_datagen.flow_from_directory (
    "/content/Dhad-Split/test",
    target_size=(32,32),
    batch_size=32,
    class_mode="categorical",
    color_mode="grayscale",
    shuffle=True,
    seed=42
)

# extract X and Y
testing_set.reset()
X_test, y_test = next(testing_set)
for i in tqdm.tqdm(range(int(testing_set.n/32)-1)): 
  img, label = next(testing_set)
  X_test = np.append(X_test, img, axis=0 )
  y_test = np.append(y_test, label, axis=0)
print(X_test.shape, y_test.shape)

In [ ]:
cnn = load_model('/content/drive/MyDrive/Colab Notebooks/EXP4-Arch2/CNN-Dhad.h5')

cnn.pop() # this will remove the last layer
cnn.summary() # check the network 

In [ ]:
model_feat = Model(inputs=cnn.input,outputs=cnn.get_layer('dense_7').output)

feat_train = model_feat.predict(X_train)
print(feat_train.shape)

feat_val = model_feat.predict(X_val)
print(feat_val.shape)

In [ ]:
feat_test = model_feat.predict(X_test)
print(feat_test.shape)

In [ ]:
svm = SVC(kernel='rbf', C=3, gamma=0.001)
svm.fit(feat_train,np.argmax(y_train,axis=1))

print('fitting done')

In [ ]:
round(svm.score(feat_val,np.argmax(y_val,axis=1)),4) *100

In [ ]:
import pickle
# now you can save it to a file
filename = '/content/drive/MyDrive/Colab Notebooks/EXP4-CNN-SVM/CNN-SVM-Dhad.pkl'
with open(filename, 'wb') as f:
    pickle.dump(svm, f)

In [ ]:
# # and later you can load it
# filename = '/content/drive/MyDrive/Colab Notebooks/EXP4-CNN-SVM/CNN-SVM-Hijjaa.pkl'
# with open(filename, 'rb') as f:
#   svm = pickle.load(f)

In [ ]:
# define variables 
predictions = svm.predict(feat_test)
class_labels = list(testing_set.class_indices.keys())  

# confusion matrix 
confusion_matrix = metrics.confusion_matrix(np.argmax(y_test,axis=1), predictions)

# FP, FN, TP, TN
FP = confusion_matrix.sum(axis=0) - numpy.diag(confusion_matrix)  
FN = confusion_matrix.sum(axis=1) - numpy.diag(confusion_matrix)
TP = numpy.diag(confusion_matrix)
TN = confusion_matrix.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# F1-Score
F1 = 2 * ((PPV*TPR)/(PPV+TPR))

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

#######################################################################################################################

per_class = confusion_matrix.diagonal()/confusion_matrix.sum(axis=0)

m_precision = precision_score(np.argmax(y_test,axis=1), predictions, average='macro')
m_recall = recall_score(np.argmax(y_test,axis=1), predictions, average='macro')
m_f1score = f1_score(np.argmax(y_test,axis=1), predictions, average='macro')

w_accuracy = accuracy_score(np.argmax(y_test,axis=1), predictions)
w_precision = precision_score(np.argmax(y_test,axis=1), predictions, average='weighted')
w_recall = recall_score(np.argmax(y_test,axis=1), predictions, average='weighted')
w_f1score = f1_score(np.argmax(y_test,axis=1), predictions, average='weighted')

report = {
    'Character': class_labels,
    'Precision':numpy.round(PPV,4) * 100 ,
    'Recall': numpy.round(TPR,4) * 100,
    'F1-Score': numpy.round(F1,4) * 100
    }
df = pd.DataFrame(report)
print(df)
print()

avg = {
    ' ': ['Macro Average', 'Weighted Average'],
    'Precision': [numpy.round(m_precision,4) * 100, numpy.round(w_precision,4) * 100],
    'Recall': [numpy.round(m_recall,4) *100, numpy.round(w_recall,4) * 100],
    'F1-Score': [numpy.round(m_f1score,4) * 100, numpy.round(w_f1score,4) * 100],
    'Accuracy': ['', numpy.round(w_accuracy,4) * 100]
}
dfa = pd.DataFrame(avg)
print(dfa)